In [10]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from scipy import stats
import keras

In [56]:
path = os.path.join(os.getcwd(), 'data')

file_list = []
sensor_position = 'Wrist_Right'

for root, dirs, files in os.walk(path):
    for file in files:
        if ('csv' in file and sensor_position in file):
            file_list.append(os.path.join(root,file))
            
dataframes = []

for file in file_list:
    with open(file) as f:
        info = [next(f).rstrip().split(': ')[1] for x in range(5)]
        user_id = info[4]
        activity = info[2]
        
    file_data = pd.read_csv(file, header=5)
    if (file_data.size):
        dataframes.append(file_data)
        file_data.insert(0, 'user_id', user_id)
        file_data.insert(1, 'activity', activity)
    

sensor_data = pd.concat(dataframes).drop('seq', axis=1)

In [57]:
df_train, df_test = train_test_split(sensor_data, test_size=0.1)

scale_columns = ['accel_x', 'accel_y', 'accel_z']

scaler = RobustScaler()

scaler = scaler.fit(df_train[scale_columns])

df_train.loc[:, scale_columns] = scaler.transform(
  df_train[scale_columns].to_numpy()
)

df_test.loc[:, scale_columns] = scaler.transform(
  df_test[scale_columns].to_numpy()
)

D:\ProgramData\Miniconda3\lib\site-packages\pandas\core\indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
D:\ProgramData\Miniconda3\lib\site-packages\pandas\core\indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [58]:
def create_dataset(X, y, time_steps=1, step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - time_steps, step):
        v = X.iloc[i:(i + time_steps)].values
        labels = y.iloc[i: i + time_steps]
        Xs.append(v)
        ys.append(stats.mode(labels)[0][0])
    return np.array(Xs), np.array(ys).reshape(-1, 1)

TIME_STEPS = 200
STEP = 40

X_train, y_train = create_dataset(
    df_train[scale_columns],
    df_train.activity,
    TIME_STEPS,
    STEP
)

X_test, y_test = create_dataset(
    df_test[scale_columns],
    df_test.activity,
    TIME_STEPS,
    STEP
)

In [59]:
    enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

    enc = enc.fit(y_train)

    y_train = enc.transform(y_train)
    y_test = enc.transform(y_test)

In [54]:
model = keras.Sequential()
model.add(
    keras.layers.Bidirectional(
      keras.layers.LSTM(
          units=128,
          input_shape=[X_train.shape[1], X_train.shape[2]]
      )
    )
)
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dense(y_train.shape[1], activation='softmax'))

model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['acc']
)

In [55]:
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.1,
    shuffle=False
)

Epoch 1/20
47/47 [==============================] - 6s 127ms/step - loss: 2.5410 - acc: 0.1382 - val_loss: 2.3726 - val_acc: 0.2866
Epoch 2/20
47/47 [==============================] - ETA: 0s - loss: 2.4690 - acc: 0.1619

KeyboardInterrupt: 

In [27]:
model.evaluate(X_test, y_test)

18/18 [==============================] - 1s 31ms/step - loss: 2.1364 - acc: 0.2541


[2.136420488357544, 0.2541133463382721]